In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from time import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_auc_score

In [2]:
datos = pd.read_csv("events_up_to_01062018.csv",low_memory=False)


In [3]:
skus_comprados = datos.loc[datos['event']=='conversion','sku']
skus_comprados = skus_comprados.round(0).astype(int)
lista_skus = skus_comprados.drop_duplicates().tolist()

In [4]:
len(lista_skus)

1037

In [5]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import sys
import warnings
import requests
from random import choice
from requests.exceptions import RequestException

In [6]:
#ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [7]:
desktop_agents = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]
def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [24]:
g = pd.Series(lista_skus,name='sku').rename_axis('sku',axis=0)

g.to_csv('lista_skus_comprados.csv')

In [20]:
g

0        2683
1        9358
2        6314
3       10924
4       10855
5        6889
6        2741
7       12550
8        6916
9        1381
10       8302
11       8415
12      10883
13       3299
14       3647
15       3635
16       3371
17      10436
18       6248
19      15759
20       9300
21       6371
22       6581
23      10953
24       6357
25       2820
26       2987
27       8541
28       7237
29       9246
        ...  
1007    15534
1008     8162
1009     2989
1010    10196
1011    13249
1012     6243
1013    10126
1014     5447
1015     3157
1016      102
1017    13178
1018     1429
1019     3289
1020    10506
1021     8105
1022     4812
1023    10799
1024     7419
1025     8568
1026    12787
1027      837
1028    12283
1029     3227
1030     3228
1031     7685
1032     9678
1033      473
1034    13753
1035    11862
1036     9384
Name: sku, Length: 1037, dtype: int64

In [8]:
proxies = [
    {"http":'http://190.15.195.64:47912',
    "https":'http://190.15.195.64:47912'},
    {"http":'http://190.16.179.24:34848',
    "https":'http://190.16.179.24:34848'},
    {"http":'http://181.167.238.28:60531',
    "https":'http://181.167.238.28:60531'},
    {"http":'http://186.56.99.103:56338',
    "https":'http://186.56.99.103:56338'},
    {"http":'http://200.81.169.242:52947',
    "https":'http://200.81.169.242:52947'},
    {"http":'http://181.47.8.154:42230',
    "https":'http://181.47.8.154:42230'}
]

In [ ]:
import time
lista_sku_precios = []
tiempos = [0,10,30,40,17,2,16,31,20,9,4,14,22,20,43,5]
for sku in lista_skus:
    try:
        time.sleep(choice(tiempos))
        raw_html = requests.get('https://www.trocafone.com.ar/comprar/checkout?trocable='+str(sku)\
                                                    ,headers=random_headers(),proxies=choice(proxies))
    except BaseHTTPError:
        continue
    html = BeautifulSoup(raw_html.text,'html.parser')
    price = html.findAll("span",{"class":'js-trocable-price-text'})
    if(not price):
        continue
    lista_sku_precios.append([sku,int(price[0].text)])

In [ ]:
print(len(lista_sku_precios))
print(len(lista_skus))

In [ ]:
pd.DataFrame(lista_sku_precios,columns=('sku','precio')).to_csv('precios_por_sku_noviembre.csv', encoding='utf-8', index=False)